In [4]:
"Implementation based on https://github.com/PetarV-/DGI"
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch.nn as nn
from models import GIC, LogReg
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import statistics
import argparse
import pickle as pkl
import networkx as nx

from scipy.sparse.linalg import eigsh

import sys
import torch.nn.functional as F
import sklearn
import sklearn.cluster
#from functional import  pairwise_distances
from layers import GCN, AvgReadout, Discriminator, Discriminator_cluster

def parse_skipgram(fname):
    with open(fname) as f:
        toks = list(f.read().split())
    nb_nodes = int(toks[0])
    nb_features = int(toks[1])
    ret = np.empty((nb_nodes, nb_features))
    it = 2
    for i in range(nb_nodes):
        cur_nd = int(toks[it]) - 1
        it += 1
        for j in range(nb_features):
            cur_ft = float(toks[it])
            ret[cur_nd][j] = cur_ft
            it += 1
    return ret

# Process a (subset of) a TU dataset into standard form
def process_tu(data, nb_nodes):
    nb_graphs = len(data)
    ft_size = data.num_features

    features = np.zeros((nb_graphs, nb_nodes, ft_size))
    adjacency = np.zeros((nb_graphs, nb_nodes, nb_nodes))
    labels = np.zeros(nb_graphs)
    sizes = np.zeros(nb_graphs, dtype=np.int32)
    masks = np.zeros((nb_graphs, nb_nodes))
       
    for g in range(nb_graphs):
        sizes[g] = data[g].x.shape[0]
        features[g, :sizes[g]] = data[g].x
        labels[g] = data[g].y[0]
        masks[g, :sizes[g]] = 1.0
        e_ind = data[g].edge_index
        coo = sp.coo_matrix((np.ones(e_ind.shape[1]), (e_ind[0, :], e_ind[1, :])), shape=(nb_nodes, nb_nodes))
        adjacency[g] = coo.todense()

    return features, adjacency, labels, sizes, masks

def micro_f1(logits, labels):
    # Compute predictions
    preds = torch.round(nn.Sigmoid()(logits))
    
    # Cast to avoid trouble
    preds = preds.long()
    labels = labels.long()

    # Count true positives, true negatives, false positives, false negatives
    tp = torch.nonzero(preds * labels).shape[0] * 1.0
    tn = torch.nonzero((preds - 1) * (labels - 1)).shape[0] * 1.0
    fp = torch.nonzero(preds * (labels - 1)).shape[0] * 1.0
    fn = torch.nonzero((preds - 1) * labels).shape[0] * 1.0

    # Compute micro-f1 score
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = (2 * prec * rec) / (prec + rec)
    return f1

"""
 Prepare adjacency matrix by expanding up to a given neighbourhood.
 This will insert loops on every node.
 Finally, the matrix is converted to bias vectors.
 Expected shape: [graph, nodes, nodes]
"""
def adj_to_bias(adj, sizes, nhood=1):
    nb_graphs = adj.shape[0]
    mt = np.empty(adj.shape)
    for g in range(nb_graphs):
        mt[g] = np.eye(adj.shape[1])
        for _ in range(nhood):
            mt[g] = np.matmul(mt[g], (adj[g] + np.eye(adj.shape[1])))
        for i in range(sizes[g]):
            for j in range(sizes[g]):
                if mt[g][i][j] > 0.0:
                    mt[g][i][j] = 1.0
    return -1e9 * (1.0 - mt)


###############################################
# This section of code adapted from tkipf/gcn #
###############################################

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def load_data(dataset_str): # {'pubmed', 'citeseer', 'cora'}
    """Load data."""
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer'  :
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    return adj, features, labels, idx_train, idx_val, idx_test

def sparse_to_tuple(sparse_mx, insert_batch=False):
    """Convert sparse matrix to tuple representation."""
    """Set insert_batch=True if you want to insert a batch dimension."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        if insert_batch:
            coords = np.vstack((np.zeros(mx.row.shape[0]), mx.row, mx.col)).transpose()
            values = mx.data
            shape = (1,) + mx.shape
        else:
            coords = np.vstack((mx.row, mx.col)).transpose()
            values = mx.data
            shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

def standardize_data(f, train_mask):
    """Standardize feature matrix and convert to tuple representation"""
    # standardize data
    f = f.todense()
    mu = f[train_mask == True, :].mean(axis=0)
    sigma = f[train_mask == True, :].std(axis=0)
    f = f[:, np.squeeze(np.array(sigma > 0))]
    mu = f[train_mask == True, :].mean(axis=0)
    sigma = f[train_mask == True, :].std(axis=0)
    f = (f - mu) / sigma
    return f

def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# Perform train-test split
    # Takes in adjacency matrix in sparse format
    # Returns: adj_train, train_edges, val_edges, val_edges_false, 
        # test_edges, test_edges_false
def mask_test_edges(adj, test_frac=.1, val_frac=0, prevent_disconnect=True, verbose=False):
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    "from https://github.com/tkipf/gae"
    
    if verbose == True:
        print('preprocessing...')

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    g = nx.from_scipy_sparse_matrix(adj)
    orig_num_cc = nx.number_connected_components(g)

    adj_triu = sp.triu(adj) # upper triangular portion of adj matrix
    adj_tuple = sparse_to_tuple(adj_triu) # (coords, values, shape), edges only 1 way
    edges = adj_tuple[0] # all edges, listed only once (not 2 ways)
    # edges_all = sparse_to_tuple(adj)[0] # ALL edges (includes both ways)
    num_test = int(np.floor(edges.shape[0] * test_frac)) # controls how large the test set should be
    num_val = int(np.floor(edges.shape[0] * val_frac)) # controls how alrge the validation set should be

    # Store edges in list of ordered tuples (node1, node2) where node1 < node2
    edge_tuples = [(min(edge[0], edge[1]), max(edge[0], edge[1])) for edge in edges]
    all_edge_tuples = set(edge_tuples)
    train_edges = set(edge_tuples) # initialize train_edges to have all edges
    test_edges = set()
    val_edges = set()

    if verbose == True:
        print('generating test/val sets...')

    # Iterate over shuffled edges, add to train/val sets
    np.random.shuffle(edge_tuples)
    for edge in edge_tuples:
        # print edge
        node1 = edge[0]
        node2 = edge[1]

        # If removing edge would disconnect a connected component, backtrack and move on
        g.remove_edge(node1, node2)
        if prevent_disconnect == True:
            if nx.number_connected_components(g) > orig_num_cc:
                g.add_edge(node1, node2)
                continue

        # Fill test_edges first
        if len(test_edges) < num_test:
            test_edges.add(edge)
            train_edges.remove(edge)

        # Then, fill val_edges
        elif len(val_edges) < num_val:
            val_edges.add(edge)
            train_edges.remove(edge)

        # Both edge lists full --> break loop
        elif len(test_edges) == num_test and len(val_edges) == num_val:
            break

    if (len(val_edges) < num_val or len(test_edges) < num_test):
        print("WARNING: not enough removable edges to perform full train-test split!")
        print("Num. (test, val) edges requested: (", num_test, ", ", num_val, ")")
        print("Num. (test, val) edges returned: (", len(test_edges), ", ", len(val_edges), ")")

    if prevent_disconnect == True:
        assert nx.number_connected_components(g) == orig_num_cc

    if verbose == True:
        print('creating false test edges...')

    test_edges_false = set()
    while len(test_edges_false) < num_test:  
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue

        false_edge = (min(idx_i, idx_j), max(idx_i, idx_j))

        # Make sure false_edge not an actual edge, and not a repeat
        if false_edge in all_edge_tuples:
            continue
        if false_edge in test_edges_false:
            continue

        test_edges_false.add(false_edge)

    if verbose == True:
        print('creating false val edges...')

    val_edges_false = set()
    while len(val_edges_false) < num_val:
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue

        false_edge = (min(idx_i, idx_j), max(idx_i, idx_j))

        # Make sure false_edge in not an actual edge, not in test_edges_false, not a repeat
        if false_edge in all_edge_tuples or \
            false_edge in test_edges_false or \
            false_edge in val_edges_false:
            continue
            
        val_edges_false.add(false_edge)

    if verbose == True:
        print('creating false train edges...')

    train_edges_false = set()
    while len(train_edges_false) < len(train_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue

        false_edge = (min(idx_i, idx_j), max(idx_i, idx_j))

        # Make sure false_edge in not an actual edge, not in test_edges_false, 
            # not in val_edges_false, not a repeat
        if false_edge in all_edge_tuples or \
            false_edge in test_edges_false or \
            false_edge in val_edges_false or \
            false_edge in train_edges_false:
            continue

        train_edges_false.add(false_edge)

    if verbose == True:
        print('final checks for disjointness...')

    # assert: false_edges are actually false (not in all_edge_tuples)
    assert test_edges_false.isdisjoint(all_edge_tuples)
    assert val_edges_false.isdisjoint(all_edge_tuples)
    assert train_edges_false.isdisjoint(all_edge_tuples)

    # assert: test, val, train false edges disjoint
    assert test_edges_false.isdisjoint(val_edges_false)
    assert test_edges_false.isdisjoint(train_edges_false)
    assert val_edges_false.isdisjoint(train_edges_false)

    # assert: test, val, train positive edges disjoint
    assert val_edges.isdisjoint(train_edges)
    assert test_edges.isdisjoint(train_edges)
    assert val_edges.isdisjoint(test_edges)

    if verbose == True:
        print('creating adj_train...')

    # Re-build adj matrix using remaining graph
    adj_train = nx.adjacency_matrix(g)

    # Convert edge-lists to numpy arrays
    train_edges = np.array([list(edge_tuple) for edge_tuple in train_edges])
    train_edges_false = np.array([list(edge_tuple) for edge_tuple in train_edges_false])
    val_edges = np.array([list(edge_tuple) for edge_tuple in val_edges])
    val_edges_false = np.array([list(edge_tuple) for edge_tuple in val_edges_false])
    test_edges = np.array([list(edge_tuple) for edge_tuple in test_edges])
    test_edges_false = np.array([list(edge_tuple) for edge_tuple in test_edges_false])

    if verbose == True:
        print('Done with train-test split!')
        print('')

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, train_edges_false, \
        val_edges, val_edges_false, test_edges, test_edges_false

'''
    pytorch (differentiable) implementation of soft k-means clustering. 
    Modified from https://github.com/bwilder0/clusternet
'''
def cluster(data, k, temp, num_iter, init, cluster_temp):
    cuda0 = False#False
    
    mu = init
    n = data.shape[0]
    d = data.shape[1]
    
    data = data / (data.norm(dim=1)[:, None] + 1e-6) #prevent zero-division loss with 1e-6
    for t in range(num_iter):
        
        mu = mu / (mu.norm(dim=1)[:, None] + 1e-6) #prevent zero-division with 1e-6
        dist = torch.mm(data, mu.transpose(0,1))

        #cluster responsibilities via softmax
        r = F.softmax(cluster_temp*dist, dim=1)
        #total responsibility of each cluster
        cluster_r = r.sum(dim=0)
        #mean of points in each cluster weighted by responsibility
        cluster_mean = r.t() @ data
        #update cluster means
        new_mu = torch.diag(1/cluster_r) @ cluster_mean
        mu = new_mu
        
    r = F.softmax(cluster_temp*dist, dim=1)
    return mu, r

class Clusterator(nn.Module):
    def __init__(self, nout, K):
        super(Clusterator, self).__init__()

        self.sigmoid = nn.Sigmoid()
        self.K = K
        self.nout = nout
        self.init =  torch.rand(self.K, nout)
        
    def forward(self, embeds, cluster_temp, num_iter=10):
        mu_init, _ = cluster(embeds, self.K, 1, num_iter, cluster_temp = torch.tensor(cluster_temp), init = self.init)
        #self.init = mu_init.clone().detach()
        mu, r = cluster(embeds, self.K, 1, 1, cluster_temp = torch.tensor(cluster_temp), init = mu_init.clone().detach())
        return mu, r
    
class GIC(nn.Module):
    def __init__(self,n_nb, n_in, n_h, activation, num_clusters, beta):
        super(GIC, self).__init__()
        self.gcn = GCN(n_in, n_h, activation)
        self.read = AvgReadout()
        self.sigm = nn.Sigmoid()
        self.disc = Discriminator(n_h)
        self.disc_c = Discriminator_cluster(n_h,n_h,n_nb,num_clusters)
        self.beta = beta
        self.cluster = Clusterator(n_h,num_clusters)
    
    def forward(self, seq1, seq2, adj, sparse, msk, samp_bias1, samp_bias2, cluster_temp):
        h_1 = self.gcn(seq1, adj, sparse)
        h_2 = self.gcn(seq2, adj, sparse)
        self.beta = cluster_temp
        Z, S = self.cluster(h_1[-1,:,:], cluster_temp)
        Z_t = S @ Z
        c2 = Z_t
        c2 = self.sigm(c2)
        c = self.read(h_1, msk)
        c = self.sigm(c) 
        c_x = c.unsqueeze(1)
        c_x = c_x.expand_as(h_1)
        ret = self.disc(c_x, h_1, h_2, samp_bias1, samp_bias2)
        ret2 = self.disc_c(c2, c2,h_1[-1,:,:], h_1[-1,:,:] ,h_2[-1,:,:], S , samp_bias1, samp_bias2)
        return ret, ret2 

    # Detach the return variables
    def embed(self, seq, adj, sparse, msk, cluster_temp):
        h_1 = self.gcn(seq, adj, sparse)
        c = self.read(h_1, msk)
        Z, S = self.cluster(h_1[-1,:,:], self.beta)
        H = S@Z
        return h_1.detach(), H.detach(), c.detach(), Z.detach()

In [5]:
def load_network_data(filename_adj, nodes_numbers):
    raw_edges = pd.read_csv(filename_adj, header=None, sep='\t')
    
    drop_self_loop = raw_edges[raw_edges[0] != raw_edges[1]]
    
    graph_np = np.zeros((nodes_numbers, nodes_numbers))
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0], drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1], drop_self_loop.iloc[i,0]]=1
        
    adj = nx.adjacency_matrix(nx.from_numpy_matrix(graph_np))
    
    features = sp.lil_matrix(np.eye(nodes_numbers))
    
    return adj, features

def get_roc_score(edges_pos, edges_neg, embeddings, adj_sparse):
    "from https://github.com/tkipf/gae"
    
    score_matrix = np.dot(embeddings, embeddings.T)
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        preds_pos.append(sigmoid(score_matrix[edge[0], edge[1]])) # predicted score
        pos.append(adj_sparse[edge[0], edge[1]]) # actual value (1 for positive)
        
    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        preds_neg.append(sigmoid(score_matrix[edge[0], edge[1]])) # predicted score
        neg.append(adj_sparse[edge[0], edge[1]]) # actual value (0 for negative)
    
    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    
    #print(preds_all, labels_all )
    
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    
    return ap_score, roc_score

In [6]:
torch.manual_seed(1234)
parser = argparse.ArgumentParser(description='Options')
parser.add_argument('--d', dest='dataset', type=str, default='wiki',help='')
parser.add_argument('--b', dest='beta', type=int, default=100,help='')
parser.add_argument('--c', dest='num_clusters', type=float, default=256,help='')
parser.add_argument('--a', dest='alpha', type=float, default=0.5,help='')
parser.add_argument('--test_rate', dest='test_rate', type=float, default=0.1,help='')
args, _ = parser.parse_known_args()
#print(args.accumulate(args.integers))

cuda0 = False

beta = args.beta
alpha = args.alpha
num_clusters = int(args.num_clusters)

dataset = args.dataset

# training params
batch_size = 1
nb_epochs = 2000
patience = 50
lr = 0.001
l2_coef = 0.0
drop_prob = 0.0
hid_units = 16
sparse = True
nonlinearity = 'prelu' # special name to separate parameters
      
torch.cuda.empty_cache()

In [8]:
roc0=[]
ap0=[]
roc1=[]
ap1=[]
roc100 = []
ap100 = []
adj_name = 'datasets/wiki.txt'
nodes_number = 2405

for m in range(10):
    print("################# m: ",m)
    adj, features = load_network_data(adj_name, nodes_number)
    adj_sparse = adj
    #print('Edges init',adj.getnnz())
    adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj, test_frac=args.test_rate, val_frac=0)
    print("数据集数量： ",len(train_edges), len(train_edges_false), len(val_edges), len(val_edges_false), len(test_edges), len(test_edges_false))
    
    adj = adj_train
    #print('Edges new',adj.getnnz())
    #ylabels = labels
    features, _ = preprocess_features(features)
    nb_nodes = features.shape[0]
    ft_size = features.shape[1]
    #nb_classes = labels.shape[1]

    adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    if sparse:
        sp_adj = sparse_mx_to_torch_sparse_tensor(adj)
    else:
        adj = (adj + sp.eye(adj.shape[0])).todense()

    features = torch.FloatTensor(features[np.newaxis])
    if not sparse:
        adj = torch.FloatTensor(adj[np.newaxis])
    #labels = torch.FloatTensor(labels[np.newaxis])
    #idx_train = torch.LongTensor(idx_train)
    #idx_val = torch.LongTensor(idx_val)
    #idx_test = torch.LongTensor(idx_test)

    b_xent = nn.BCEWithLogitsLoss()
    b_bce = nn.BCELoss()
    #xent = nn.CrossEntropyLoss()
    
    all_accs = []

    for beta in [args.beta]:
        #print("bata: ",beta)
        for K in [int(args.num_clusters)]:
            #K = int(Kr * nb_nodes)
            for alpha in [args.alpha]:
                #print(m, alpha)
                model = GIC(nb_nodes,ft_size, hid_units, nonlinearity, num_clusters, beta)
                optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)
                cnt_wait = 0
                best = 1e9
                best_t = 0
                val_best = 0

                for epoch in range(nb_epochs):
                    model.train()
                    optimiser.zero_grad()

                    idx = np.random.permutation(nb_nodes)
                    shuf_fts = features[:, idx, :]

                    lbl_1 = torch.ones(batch_size, nb_nodes)
                    lbl_2 = torch.zeros(batch_size, nb_nodes)
                    lbl = torch.cat((lbl_1, lbl_2), 1)
                    
                    logits, logits2  = model(features, shuf_fts, sp_adj if sparse else adj, sparse, None, None, None, beta) 

                    loss = alpha* b_xent(logits, lbl)  + (1-alpha)*b_xent(logits2, lbl) 

                    if loss < best:
                        best = loss
                        best_t = epoch
                        cnt_wait = 0
                        torch.save(model.state_dict(), dataset+'-link.pkl')
                        
                    else:
                        cnt_wait += 1

                    if cnt_wait == patience:
                        #print('Early stopping!')
                        break

                    loss.backward()
                    optimiser.step()

                model.load_state_dict(torch.load(dataset+'-link.pkl'))

                embeds, _,_, S= model.embed(features, sp_adj if sparse else adj, sparse, None, beta)
                embs = embeds[0, :]
                embs = embs / embs.norm(dim=1)[:, None]
                ap_score, auc_score = get_roc_score(test_edges, test_edges_false, embs.cpu().detach().numpy(), adj_sparse)
                #print(beta, K, alpha, sc_roc, sc_ap,flush=True)
                #print('Dataset',args.dataset)
                #print('alpha, beta, K:',alpha,beta,K)
                print('ap: ', ap_score, '    auc: ', auc_score)

################# m:  0
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8785712135748397     auc:  0.8332746461834865
################# m:  1
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8829726084188272     auc:  0.8447688905002007
################# m:  2
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8666525520316688     auc:  0.8188480295634346
################# m:  3
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8735887889973848     auc:  0.8278409357386876
################# m:  4
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8870067970895352     auc:  0.8444264453975007
################# m:  5
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8832334389537413     auc:  0.8464773937843237
################# m:  6
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8889623419385022     auc:  0.8497559334197387
################# m:  7
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8873562277611234     auc:  0.8445232233613071
################# m:  8
数据集数量：  10437 10437 0 0 1159 1159
ap:  0.8739298740193978     auc:  0.8260929768231664
#

In [1]:
AP = [0.8786, 0.8830, 0.8667, 0.8736, 0.8870, 0.8832, 0.8890, 0.8874, 0.8739, 0.8895]
AUC = [0.8333, 0.8448, 0.8188, 0.8278, 0.8444, 0.8465, 0.8498, 0.8445, 0.8261, 0.8546]

In [2]:
import numpy as np

In [3]:
np.mean(AP), np.std(AP), np.mean(AUC), np.std(AUC)

(0.8811899999999999,
 0.007328772066315047,
 0.8390600000000001,
 0.011142908058491753)